In [1]:
import os
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import albumentations as albu
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import KFold
import pytorch_lightning as pl
from torch.utils.data import DataLoader
import timm
from torchmetrics import Metric
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import random

In [2]:
class NMAE(Metric):
    def __init__(self):
        super().__init__()
        self.add_state("nmae", default=torch.tensor(0.), dist_reduce_fx="sum")
        self.add_state("total", default=torch.tensor(0), dist_reduce_fx="sum")
    def update(self, preds: torch.Tensor, target: torch.Tensor):
        assert preds.shape == target.shape
        nmae = torch.sum(torch.abs(target-preds) / (torch.abs(target)))
        n_obs = target.numel()
        self.nmae += nmae
        self.total += n_obs
    def compute(self):
        return self.nmae / self.total

In [90]:
class NMAE(Metric):
    def __init__(self):
        super().__init__()
        self.add_state("mae", default=torch.tensor(0.), dist_reduce_fx="sum")
        self.add_state("deno", default=torch.tensor(0.), dist_reduce_fx="sum")
    def update(self, preds: torch.Tensor, target: torch.Tensor):
        assert preds.shape == target.shape
        mae = torch.mean(torch.abs(target-preds))
        deno = torch.mean(torch.abs(target))
        self.mae += mae
        self.deno += deno
    def compute(self):
        return self.mae / self.deno
    


In [94]:
class NMAE(Metric):
    def __init__(self):
        super().__init__()
        self.add_state("mae", default=[], dist_reduce_fx=None)
        self.add_state("deno", default=[], dist_reduce_fx=None)
        
    def update(self, preds: torch.Tensor, target: torch.Tensor):
        assert preds.shape == target.shape
        mae = torch.abs(target-preds)
        deno = torch.abs(target)
        self.mae.append(mae)
        self.deno.append(deno)
    def compute(self):
        return torch.mean(torch.cat(self.mae)) / torch.mean(torch.cat(self.deno))

In [113]:
#processing outlier
'''
중복데이터 삭제
case59, case11, case12, case13, case14,
이상치 리스트
확인 : CASE05_21, CASE10_41, case16, case22, case23, case45-16, case45-17,
CASE73_04, case73-10,CASE73_11, CASE73_14
'''
preprocessing_train_imgs = glob("D:/DACON_GROWTH/PREPROCESSING-TRAIN_224/*")
preprocessing_test_imgs = glob("D:/DACON_GROWTH/PREPROCESSING-TEST_224/*")
augmetation_imgs = glob("D:/DACON_GROWTH/AUGMENTATION/SIZE224/*")

outlier_imgs = []
outlier_path = []
train_path = []
train_imgs = []
for path in tqdm(preprocessing_train_imgs):
    img_file = os.path.basename(path)
    img_name = img_file.split(".")[0]
    case = img_name.split("_")[0]
    if case in ["CASE59", "CASE11", "CASE12", "CASE13", "CASE14"]:
        continue
    elif img_name in ["CASE05_21", "CASE10_41", "CASE45_16", "CASE45_17",
                     "CASE73_04", "CASE73_10", "CASE73_11", "CASE73_14"]:        
        continue
    elif case in ["CASE16", "CASE22", "CASE23"]:
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        outlier_path.append(path)
        outlier_imgs.append(img)
    else:
        img = cv2.imread(path, cv2.IMREAD_COLOR)
        train_path.append(path)
        train_imgs.append(img)
        
test_path = []
test_imgs = []
for path in tqdm(preprocessing_test_imgs):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    test_path.append(path)
    test_imgs.append(img)

augment_path = []
augment_imgs = []
for path in tqdm(augmetation_imgs):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    augment_path.append(path)
    augment_imgs.append(img)
    
augment_path = augment_path + outlier_path
augment_imgs = augment_imgs + outlier_imgs

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 492/492 [00:02<00:00, 169.92it/s]


In [114]:
augment_path

['D:/DACON_GROWTH/AUGMENTATION/SIZE224\\augment_data1.png',
 'D:/DACON_GROWTH/AUGMENTATION/SIZE224\\augment_data10.png',
 'D:/DACON_GROWTH/AUGMENTATION/SIZE224\\augment_data100.png',
 'D:/DACON_GROWTH/AUGMENTATION/SIZE224\\augment_data101.png',
 'D:/DACON_GROWTH/AUGMENTATION/SIZE224\\augment_data102.png',
 'D:/DACON_GROWTH/AUGMENTATION/SIZE224\\augment_data103.png',
 'D:/DACON_GROWTH/AUGMENTATION/SIZE224\\augment_data104.png',
 'D:/DACON_GROWTH/AUGMENTATION/SIZE224\\augment_data105.png',
 'D:/DACON_GROWTH/AUGMENTATION/SIZE224\\augment_data106.png',
 'D:/DACON_GROWTH/AUGMENTATION/SIZE224\\augment_data107.png',
 'D:/DACON_GROWTH/AUGMENTATION/SIZE224\\augment_data108.png',
 'D:/DACON_GROWTH/AUGMENTATION/SIZE224\\augment_data109.png',
 'D:/DACON_GROWTH/AUGMENTATION/SIZE224\\augment_data11.png',
 'D:/DACON_GROWTH/AUGMENTATION/SIZE224\\augment_data110.png',
 'D:/DACON_GROWTH/AUGMENTATION/SIZE224\\augment_data111.png',
 'D:/DACON_GROWTH/AUGMENTATION/SIZE224\\augment_data112.png',
 'D:/DACON_G

In [3]:
#efficientV2_l => train 384, test 480, augment 384
preprocessing_train_imgs = glob("D:/DACON_GROWTH/PREPROCESSING-TRAIN_224/*")
preprocessing_test_imgs = glob("D:/DACON_GROWTH/PREPROCESSING-TEST_224/*")
augmetation_imgs = glob("D:/DACON_GROWTH/AUGMENTATION/SIZE224/*")

train_path = []
train_imgs = []
for path in tqdm(preprocessing_train_imgs):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    train_path.append(path)
    train_imgs.append(img)

test_path = []
test_imgs = []
for path in tqdm(preprocessing_test_imgs):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    test_path.append(path)
    test_imgs.append(img)

augment_path = []
augment_imgs = []
for path in tqdm(augmetation_imgs):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    augment_path.append(path)
    augment_imgs.append(img)


train_df = pd.read_parquet("D:/DACON_GROWTH/train.parquet")
test_df = pd.read_parquet("D:/DACON_GROWTH/test.parquet")

train_df["무게"] = train_df["무게"].apply(lambda x: 0.001 if x<0.001 else x)

img2weight = {}
for i in tqdm(train_df["이미지"].unique()):
    temp = train_df[train_df["이미지"]==i]
    weight = temp["무게"].values[0]
    img2weight[i] = weight
    
train_df["시간2"] = train_df["시간"].apply(lambda x: x.split(" ")[-1])
golden_time = ['17:40', '17:41', '17:42', '17:43', '17:44', '17:45', '17:46', '17:47', '17:48', '17:49', '17:50',
               '17:51', '17:52', '17:53', '17:54', '17:55', '17:56', '17:57', '17:58', '17:59', '18:00', '18:01',
               '18:02', '18:03', '18:04', '18:05']
golden_dict ={}
for e, i in enumerate(train_df["시간2"].unique()):
       golden_dict[i] = e
train_df["순서"] = train_df["시간2"].apply(lambda x : golden_dict[x])
start = golden_dict['17:40']
end = golden_dict["18:05"]

train_golden_df = train_df[(train_df["순서"] > 894) & (train_df["순서"] < 919)]

features = ['내부온도관측치', '외부온도관측치', '내부습도관측치', '외부습도관측치', 'CO2관측치', 'EC관측치',
       '최근분무량', '화이트 LED동작강도', '레드 LED동작강도', '블루 LED동작강도', '냉방온도', '냉방부하',
       '난방온도', '난방부하', '총추정광량', '백색광추정광량', '적색광추정광량', '청색광추정광량']

img2data = {}
for i in tqdm(train_golden_df["이미지"].unique()):
       temp = train_golden_df[train_golden_df["이미지"]==i]
       temp_mean = temp[features].mean(axis=0)
       img2data[temp["이미지"].values[0]] = temp_mean[["EC관측치",'내부습도관측치', '외부습도관측치']].values

test_df["시간2"] = test_df["시간"].apply(lambda x: x.split(" ")[-1])
test_df["순서"] = test_df["시간2"].apply(lambda x : golden_dict[x])
test_golden_df = test_df[(test_df["순서"] > 894) & (test_df["순서"] < 919)]
img2test = {}
for i in tqdm(test_golden_df["이미지"].unique()):
       temp = test_golden_df[test_golden_df["이미지"]==i]
       temp_mean = temp[features].mean(axis=0)
       img2test[temp["이미지"].values[0]] = temp_mean[["EC관측치",'내부습도관측치', '외부습도관측치']].values

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 460/460 [00:01<00:00, 318.05it/s]


In [ ]:
train_key = []
for k, v in img2weight.items():
    if v < 0.5:
        train_key.append(k)

In [21]:
train_path

array(['D:/DACON_GROWTH/PREPROCESSING-TRAIN_224\\CASE01_01.png',
       'D:/DACON_GROWTH/PREPROCESSING-TRAIN_224\\CASE01_02.png',
       'D:/DACON_GROWTH/PREPROCESSING-TRAIN_224\\CASE01_03.png', ...,
       'D:/DACON_GROWTH/PREPROCESSING-TRAIN_224\\CASE74_27.png',
       'D:/DACON_GROWTH/PREPROCESSING-TRAIN_224\\CASE75_01.png',
       'D:/DACON_GROWTH/PREPROCESSING-TRAIN_224\\CASE75_02.png'],
      dtype='<U53')

In [34]:
main_path = "D:/DACON_GROWTH"
train_imgs = glob(main_path+"/train/*/*/*.png") + glob(main_path+"/train/*/*/*.jpg")
train_imgs = sorted(train_imgs)
test_imgs = glob(main_path+"/test/image/*.png") + glob(main_path+"/test/image/*.jpg")
test_imgs = sorted(test_imgs)
train_data = glob(main_path+"/train/*/meta/*.csv")
train_label = glob(main_path+"/train/*/*.csv")
test_data = glob(main_path+"/test/meta/*.csv")

train_df = []
for i in tqdm(train_data):
    name = i.split("\\")[-1].split(".")[0]
    df = pd.read_csv(i)
    df["이미지"] = name
    case = name.split("_")[0]
    label = pd.read_csv(f"D:/DACON_GROWTH/train/{case}/label.csv")
    label_name = [i.split(".")[0] for i in label.img_name]
    label.img_name = label_name
    leaf_weight = label[label.img_name == name].leaf_weight.values[0]
    df["무게"] = leaf_weight
    df = df.replace([np.inf, -np.inf], np.nan)
    train_df.append(df)

train_df = pd.concat(train_df, ignore_index=True)

train_df["시간2"] = train_df["시간"].apply(lambda x: x.split(" ")[-1])
golden_time = ['17:40', '17:41', '17:42', '17:43', '17:44', '17:45', '17:46', '17:47', '17:48', '17:49', '17:50',
               '17:51', '17:52', '17:53', '17:54', '17:55', '17:56', '17:57', '17:58', '17:59', '18:00', '18:01',
               '18:02', '18:03', '18:04', '18:05']
golden_dict ={}
for e, i in enumerate(train_df["시간2"].unique()):
       golden_dict[i] = e
train_df["순서"] = train_df["시간2"].apply(lambda x : golden_dict[x])

train_df2 = train_df[(train_df["순서"] > 880) & (train_df["순서"] < 930)]

train_dfx = train_df2.copy()
train_dfx.drop(columns=["시간", "시간2"], inplace=True)
train_dfx.fillna(0., inplace=True)

img2data = {}
for i in tqdm(train_dfx['이미지'].unique()):
    temp = train_dfx[train_dfx["이미지"]==i]
    temp.drop(columns = ["무게", "이미지"], inplace=True)
    img2data[i] = temp.values




 33%|██████████████████████████████████████████████████████████████▏                                                                                                                              | 524/1592 [00:08<00:13, 81.65it/s]


 66%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                                | 1046/1592 [00:16<00:08, 61.71it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1592/1592 [00:24<00:00, 66.09it/s]

  0%|                                                                                                                                                                                                       | 0/1592 [00:00<?, ?it/s]D:\meta2\venv\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,



 61%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                                                        | 975/1592 [00:07<00:04, 130.31it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1592/1592 [00:12<00:00, 131.35it/s]


In [45]:
img2data = {}
for i in tqdm(train_dfx['이미지'].unique()):
    temp = train_dfx[train_dfx["이미지"]==i]
    temp.drop(columns = ["무게", "이미지"], inplace=True)
    img2data[i] = np.array(temp.values, dtype=np.float32)


 58%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                              | 929/1592 [00:07<00:05, 123.43it/s]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1592/1592 [00:12<00:00, 123.31it/s]


In [50]:
class PYTORCH_DATASET(nn.Module):
    def __init__(self, path: list, img2weight: dict, imgs: np.array, img2data: dict,  \
                 transform=None):
        super(PYTORCH_DATASET, self).__init__()
        self.path = path
        self.imgs = imgs

        self.transform = transform
        self.img2weight = img2weight
        self.img2data = img2data

    def __len__(self):
        return len(self.path)

    def __getitem__(self, idx):
        before_img = self.imgs[idx]
        img_path = self.path[idx]
        img_path = img_path.split("\\")[-1].split(".")[0]
        df_index = self.img2data[img_path]
        df_label = self.img2weight[img_path]

        after_img = self.transform(image=before_img)["image"]

        return {"사진": after_img, "정보": df_index, "상표": df_label}


class PYTORCH_PREDICTSET(nn.Module):
    def __init__(self, path: list,  imgs: np.array, img2data: dict, \
                 transform=None):
        super(PYTORCH_PREDICTSET, self).__init__()
        self.path = path
        self.imgs = imgs

        self.transform = transform
        self.img2data = img2data

    def __len__(self):
        return len(self.path)

    def __getitem__(self, idx):
        before_img = self.imgs[idx]
        img_path = self.path[idx]
        img_path = img_path.split("\\")[-1].split(".")[0]
        df_index = self.img2data[img_path]
        after_img = self.transform(image=before_img)["image"]
        return {"사진": after_img, "정보": df_index}

class PYTORCH_AUGMENTSET(nn.Module):
    def __init__(self, aug_imgs: np.array, transform_ori = None, transform_aug=None):
        super(PYTORCH_AUGMENTSET, self).__init__()
        self.aug_imgs = aug_imgs
        self.transform_ori = transform_ori
        self.transform_aug = transform_aug

    def __len__(self):
        return len(self.aug_imgs)

    def __getitem__(self, idx):
        before_img = self.aug_imgs[idx]
        after_img = self.transform_aug(image=before_img)["image"]
        before_img = self.transform_ori(image=before_img)["image"]
        temp_data = torch.FloatTensor(torch.zeros([49,19]))

        return {"원본": before_img, "확장": after_img, "정보": temp_data}

In [51]:

image_size = 224

train_mode= albu.Compose([
    albu.RandomResizedCrop(height=image_size, width=image_size,
                           scale=(0.25, 1.0), ratio=(0.75, 1.3333333333333333),
                           interpolation=1, p=0.9),
    albu.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1,
                          rotate_limit=30, interpolation=1, border_mode=0, value=0, p=0.25),
    albu.HorizontalFlip(p=0.5),
    albu.VerticalFlip(p=0.5),
    albu.OneOf([
        albu.MotionBlur(p=.2),
        albu.MedianBlur(blur_limit=3, p=0.1),
        albu.Blur(blur_limit=3, p=0.1),
    ], p=0.25),
    albu.OneOf([
        albu.CLAHE(clip_limit=2),
        albu.Sharpen(),
        albu.Emboss(),
        albu.RandomBrightnessContrast(),
    ], p=0.25),
    albu.CoarseDropout(max_holes=8, max_height=8, max_width=8, fill_value=0, p=0.25),
    albu.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD),
    ToTensorV2(),
])

test_mode = albu.Compose([
    #                 albu.Resize(self.image_size, self.image_size),
    albu.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD),
    ToTensorV2(),
])

import torch
import numpy as np
from torch import nn
from einops.layers.torch import Rearrange


class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout=0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)


class MixerBlock(nn.Module):

    def __init__(self, dim, num_sequence, token_dim, channel_dim, dropout=0.):
        super().__init__()

        self.token_mix = nn.Sequential(
            nn.LayerNorm(dim),
            Rearrange('b n d -> b d n'),
            FeedForward(num_sequence, token_dim, dropout),
            Rearrange('b d n -> b n d')
        )

        self.channel_mix = nn.Sequential(
            nn.LayerNorm(dim),
            FeedForward(dim, channel_dim, dropout),
        )

    def forward(self, x):
        x = x + self.token_mix(x)

        x = x + self.channel_mix(x)

        return x

class MLPMixer(nn.Module):

    def __init__(self):
        super().__init__()

        self.dim = 19
        self.num_sequence = 49
        self.token_dim = 64
        self.channel_dim = 128
        self.mixer_blocks = nn.ModuleList([])
        self.depth = 1
        for _ in range(self.depth):
            self.mixer_blocks.append(MixerBlock(self.dim, self.num_sequence, self.token_dim, self.channel_dim))
        self.layer_norm = nn.LayerNorm(self.dim)

    def forward(self, x):
        for mixer_block in self.mixer_blocks:
            x = mixer_block(x)

        x = self.layer_norm(x)

        x = x.mean(dim=1)

        return x
    
class CNN_RNN(nn.Module):
    def __init__(self):
        super(CNN_RNN, self).__init__()
        self.model = timm.create_model("efficientnet_b0", pretrained=True, \
                                       num_classes=1)
        self.meta = MLPMixer()
        self.avg = nn.AdaptiveAvgPool2d((1, 1))
        self.linear = nn.Linear(1299, 256)
        self.output = nn.Linear(256, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.9)
        self.dropout1 = nn.Dropout(0.65)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0)

    def forward(self,  x, x2, training):
        x = self.model.forward_features(x)
        x = self.avg(x)
        x = x.flatten()
        x = x.view(-1,  1280)
        x = self.dropout(x)
        x2 = self.meta(x2)
        if training:
            c = random.choice([0, 0, 0, 1, 2, 3])
            if c == 0:
                pass
            elif c == 1:
                x2 = self.dropout1(x2)
            elif c == 2:
                x2 = self.dropout2(x2)
            elif c == 3:
                x2 = self.dropout3(x2)
        x = torch.cat([x, x2], dim=-1)
        x = self.linear(x)
        x = self.relu(x)
        x = self.output(x)
        return x


In [95]:
from pytorch_lightning.trainer.supporters import CombinedLoader
from adabelief_pytorch import AdaBelief

class LIGHTNING_MODEL(pl.LightningModule):
    def __init__(self, train_datasest, val_dataset, augment_dataset, predict_dataset):
        super(LIGHTNING_MODEL, self).__init__()
        self.model = CNN_RNN()
        self.train_nmae = NMAE()
        self.valid_nmae = NMAE()
        self.augment_dataset= augment_dataset
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.predict_dataset = predict_dataset
        self.bestnmae = 10.
        
    def forward(self, x):
        output = self.model(x)
        return output
    
    def train_dataloader(self):
        loaders = {"a": DataLoader(self.train_dataset, shuffle=True, batch_size=32, num_workers=0), \
                   "b": DataLoader(self.augment_dataset, shuffle=True, batch_size=8, num_workers=0)}
        combined_loaders = CombinedLoader(loaders, mode="max_size_cycle")
        return combined_loaders
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset, shuffle=False, batch_size=32, num_workers=0)
    
    def predict_dataloader(self):
        return DataLoader(self.predict_dataset, shuffle=False, batch_size=32, num_workers=0)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        ori_batch = train_batch["a"]
        aug_batch = train_batch["b"]
        img = ori_batch["사진"]
        data = ori_batch["정보"]
        label = ori_batch["상표"]
        
        ori = aug_batch["원본"]
        ext = aug_batch["확장"]
        temp_data = aug_batch["정보"]

        pred = self.model(img, data, training=True)
        pred = pred.view(-1)
        label = label.view(-1)
        
        train_loss = nn.L1Loss()(pred, label)
        self.log("train_loss", train_loss)
        
        train_nmae = self.train_nmae(pred, label)
        self.log('train_NMAE', train_nmae)
        
        with torch.no_grad():
            ori_out = self.model(ori, temp_data, training=False)
            ext_out = self.model(ext, temp_data, training=False)
        con_loss = nn.L1Loss()(ori_out, ext_out)
        con_loss = 0.25 * con_loss 
        self.log('con_loss', con_loss)
        loss = train_loss + con_loss
        self.log('loss', loss)
        return train_loss

    def training_epoch_end(self, outputs):
        train_nmae = self.train_nmae.compute()
        self.log("epoch_train_nmae", train_nmae)
        self.train_nmae.reset()
        print(f"training nmae: {train_nmae:.4}")
        
        
    def validation_step(self, val_batch, batch_idx):
        img = val_batch["사진"]
        data = val_batch["정보"]
        label = val_batch["상표"]

        pred = self.model(img, data, training=False)
        pred = pred.view(-1)
        label = label.view(-1)
        loss = nn.L1Loss()(pred, label)
        self.log("val_loss", loss)
        self.valid_nmae.update(pred, label)
        return loss

    def validation_epoch_end(self, outputs):
        val_nmae = self.valid_nmae.compute()
        self.log("val_nmae", val_nmae)
        ext = val_nmae.item()
        if ext < self.bestnmae:
            self.bestnmae = ext
        self.valid_nmae.reset()
        print(f"validation nmae: {val_nmae:.4}")

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        img = batch["사진"]
        data = batch["정보"]
        pred = self.model(img, data, training=False).unsqueeze(0)
        return pred


In [71]:
train_path = np.array(train_path)
train_imgs = np.array(train_imgs)

In [28]:
#########################################################
'''validation 성능평가용'''
#########################################################
path_train = []
path_test = []
img_train = []
img_test = []
path_valid = []
img_valid = []

for img, path, key in zip(train_imgs, train_path, img2weight.keys()):
    if (len(path_train) < 900) or (key in train_key):
        path_train.append(path)
        img_train.append(img)
for img, path, key in zip(train_imgs, train_path, img2weight.keys()):
    if (path not in path_train) and (len(path_valid)<200):
        path_valid.append(path)
        img_valid.append(img)
for img, path, key in zip(train_imgs, train_path, img2weight.keys()):
    if (path not in path_train) and (path not in path_valid):
        path_test.append(path)
        img_test.append(img)
        
path_train = np.array(path_train)
path_test = np.array(path_test)
img_train = np.array(img_train)
img_test = np.array(img_test)
path_valid = np.array(path_valid)
img_valid = np.array(img_valid)

train_dataset = PYTORCH_DATASET(path=path_train, img2weight=img2weight, imgs=img_train, img2data=img2data,\
                                            transform=train_mode)

valid_dataset = PYTORCH_DATASET(path=path_valid, img2weight=img2weight, imgs=img_valid, img2data=img2data,\
                                            transform=test_mode)

augment_dataset = PYTORCH_AUGMENTSET(aug_imgs=augment_imgs, transform_ori=test_mode, transform_aug=train_mode)

test_dataset = PYTORCH_PREDICTSET(path=path_test, imgs=img_test, img2data=img2data, \
                                    transform=test_mode)


In [96]:
# #########################################################
# '''이상치 검증용'''
# #########################################################

kf = KFold(n_splits=5, shuffle=False)

for fold, (ti, vi) in enumerate(kf.split(train_path)):                                                                                                   
    trainset_path, validset_path = train_path[ti], train_path[vi]
    train_imgset, valid_imgset = train_imgs[ti], train_imgs[vi]

    train_dataset = PYTORCH_DATASET(path=trainset_path, img2weight=img2weight, imgs=train_imgset, img2data=img2data,\
                                    transform=train_mode)

    valid_dataset = PYTORCH_DATASET(path=validset_path, img2weight=img2weight, imgs=valid_imgset, img2data=img2data,\
                                    transform=test_mode)

    augment_dataset = PYTORCH_AUGMENTSET(aug_imgs=augment_imgs, transform_ori=test_mode, transform_aug=train_mode)

    test_dataset = PYTORCH_PREDICTSET(path=validset_path, imgs=valid_imgset, img2data=img2data, \
                            transform=test_mode)


    model_name = "efficientnet_B0"

    model = LIGHTNING_MODEL(train_dataset, valid_dataset, augment_dataset, test_dataset)

    check_path =  f"D:/DACON_GROWTH/outlier_{fold}/{model_name}"
    if not os.path.exists(check_path):
        os.makedirs(check_path)
    checkpoint_callback = ModelCheckpoint(dirpath=check_path, save_top_k=1, monitor="val_nmae", \
                                          save_weights_only=True)
    print(1)
    trainer = pl.Trainer(
        devices=[0],
        accelerator='gpu',
        max_epochs=70,
        callbacks=[checkpoint_callback],
        precision=16
    )

    trainer.fit(model)

    bestscore = model.bestnmae
    checkpoint_ = os.listdir(check_path)[0]
    file_name = os.path.basename(checkpoint_).split(".ckpt")[0]
    src = os.path.join(check_path, checkpoint_)
    score_name = file_name + f"_{bestscore : .4f}.ckpt"
    dst = os.path.join(check_path, score_name)
    os.rename(src, dst)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type               | Params
-------------------------------------------------------
0 | model           | CNN_RNN            | 4.4 M 
1 | train_nmae      | NMAE               | 0     
2 | valid_nmae      | NMAE               | 0     
3 | augment_dataset | PYTORCH_AUGMENTSET | 0     
4 | train_dataset   | PYTORCH_DATASET    | 0     
5 | val_dataset     | PYTORCH_DATASET    | 0     
6 | predict_dataset | PYTORCH_PREDICTSET | 0     
-------------------------------------------------------
4.4 M     Trainable params
0         Non-trainable params
4.4 M     Total params
8.707     Total estimated model params size (MB)


1


Sanity Checking: 0it [00:00, ?it/s]

validation nmae: 1.001


Training: 0it [00:00, ?it/s]

...

validation nmae: 0.115
training nmae: 0.1141


Validation: 0it [00:00, ?it/s]

validation nmae: 0.1894
training nmae: 0.1138


Validation: 0it [00:00, ?it/s]

validation nmae: 0.1502
training nmae: 0.1216


In [103]:
kf = KFold(n_splits=5, shuffle=False)
fold1 = {}
fold2 = {}
fold3 = {}
fold4 = {}
fold5 = {}
for fold, (ti, vi) in enumerate(kf.split(train_path)):                                                                                                   
    trainset_path, validset_path = train_path[ti], train_path[vi]
    train_imgset, valid_imgset = train_imgs[ti], train_imgs[vi]

    valid_dataset = PYTORCH_DATASET(path=validset_path, img2weight=img2weight, imgs=valid_imgset, img2data=img2data,\
                                    transform=test_mode)

    model_name = "efficientnet_B0"
    
    if fold == 0:
        check_path = r"D:\DACON_GROWTH\outlier_0\efficientnet_B0/epoch=69-step=4340_ 0.0751.ckpt"
    elif fold == 1:
        check_path = r"D:\DACON_GROWTH\outlier_1\efficientnet_B0/epoch=47-step=2976_ 0.1439.ckpt"
    elif fold == 2:
        check_path = r"D:\DACON_GROWTH\outlier_2\efficientnet_B0/epoch=68-step=4278_ 0.0550.ckpt"
    elif fold == 3:
        check_path = r"D:\DACON_GROWTH\outlier_3\efficientnet_B0/epoch=58-step=3658_ 0.0754.ckpt"
    elif fold == 4:
        check_path = r"D:\DACON_GROWTH\outlier_4\efficientnet_B0/epoch=12-step=806_ 0.0909.ckpt"
        
    
    trainer = pl.Trainer(
        devices=[0],
        accelerator='gpu',
        max_epochs=70,
        callbacks=[checkpoint_callback],
        precision=16
    )
    

    model = LIGHTNING_MODEL.load_from_checkpoint(checkpoint_path=check_path,
                              train_datasest = None, val_dataset = None, \
                    augment_dataset= None, predict_dataset = valid_dataset)
    trainer.predict(model)
    
    prediction = trainer.predict(model)
    
    sub_list = []
    for p in prediction:
        p = p.view(-1)
        for sample in p:
            sample = sample.cpu().numpy()
            sub_list.append(sample)
    #true value
    valid_true = []
    for p in validset_path:
        p = p.split("\\")[-1].split(".")[0]
        w = img2weight[p]
        valid_true.append(w)
    
    for i, j, k in zip(validset_path, sub_list, valid_true):
        i = i.split("\\")[-1].split(".")[0]
        result = np.abs(j-k) / j
        result2 = np.abs(j-k)
        if fold == 0:
            fold1[i] = [result, result2]
        elif fold == 1:
            fold2[i] = [result, result2]
        elif fold == 2:
            fold3[i] = [result, result2]
        elif fold == 3:
            fold4[i] = [result, result2]
        elif fold == 4:
            fold5[i] = [result, result2]
    print(fold)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

0


...

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

4


In [104]:
fold1

{'CASE01_01': [0.23644933228593873, 9.40625],
 'CASE01_02': [0.2232885476647473, 10.90625],
 'CASE01_03': [0.2037519541427827, 12.21875],
 'CASE01_04': [0.18072289156626506, 13.125],
 'CASE01_05': [0.1856936416184971, 16.0625],
 'CASE01_06': [0.1799163179916318, 18.8125],
 'CASE01_07': [0.2936302433972035, 35.4375],
 'CASE01_08': [0.13004846526655897, 20.125],
 'CASE01_09': [0.034482758620689655, 6.25],
 'CASE02_01': [0.029301745635910224, 5.875],
 'CASE02_02': [0.011214953271028037, 2.25],
 'CASE02_03': [0.021414665801427646, 4.125],
 'CASE02_04': [0.020471464019851116, 4.125],
 'CASE02_05': [0.036101083032490974, 7.5],
 'CASE02_06': [0.021844660194174758, 4.5],
 'CASE02_07': [0.03128834355828221, 6.375],
 'CASE02_08': [0.09433962264150944, 19.375],
 'CASE02_09': [0.07788595271210014, 14.0],
 'CASE02_10': [0.012028869286287089, 1.875],
 'CASE02_11': [0.0008764241893076249, 0.125],
 'CASE03_01': [0.07800891530460624, 0.8203125],
 'CASE03_02': [0.15623205054566341, 2.125],
 'CASE03_03':

In [105]:
fold2

{'CASE14_39': [0.023943661971830985, 1.0625],
 'CASE14_40': [0.14839133289560077, 7.0625],
 'CASE14_41': [0.06377998829724986, 3.40625],
 'CASE15_01': [0.8219954648526077, 1.06201171875],
 'CASE15_02': [0.694920091324201, 1.18896484375],
 'CASE15_03': [0.45066273932253315, 0.896484375],
 'CASE15_04': [0.4155701754385965, 1.1103515625],
 'CASE15_05': [0.39959839357429716, 1.166015625],
 'CASE15_06': [0.4305312317571512, 1.4404296875],
 'CASE15_07': [0.524505588993981, 2.3828125],
 'CASE15_08': [0.5319289005924951, 3.15625],
 'CASE15_09': [0.26952526799387444, 2.0625],
 'CASE15_10': [0.35448666127728373, 3.42578125],
 'CASE15_11': [0.10771113831089352, 1.375],
 'CASE16_01': [0.3723916532905297, 58.0],
 'CASE16_02': [0.4024466750313676, 80.1875],
 'CASE16_03': [0.4582463465553236, 109.75],
 'CASE16_04': [0.26504941599281223, 73.75],
 'CASE17_01': [0.12841328413284134, 43.5],
 'CASE17_02': [0.2228956228956229, 82.75],
 'CASE17_03': [0.006464124111182934, 2.5],
 'CASE18_01': [0.552648790058

In [106]:
fold3

{'CASE36_19': [0.02525523911875336, 2.9375],
 'CASE36_20': [0.024472573839662448, 3.625],
 'CASE36_21': [0.009292351679771264, 1.625],
 'CASE36_22': [0.03003003003003003, 6.25],
 'CASE36_23': [0.01457286432160804, 3.625],
 'CASE36_24': [0.02145922746781116, 6.25],
 'CASE36_25': [0.035483870967741936, 11.0],
 'CASE36_26': [0.01098901098901099, 3.75],
 'CASE36_27': [0.010744985673352435, 3.75],
 'CASE37_01': [0.856339928057554, 0.4649658203125],
 'CASE37_02': [0.24285714285714285, 0.15771484375],
 'CASE37_03': [0.0494579945799458, 0.0712890625],
 'CASE37_04': [0.2808896210873147, 0.666015625],
 'CASE37_05': [0.07994670219853431, 0.234375],
 'CASE37_06': [0.3985728848114169, 1.52734375],
 'CASE37_07': [0.010503040353786623, 0.07421875],
 'CASE37_08': [0.058774834437086095, 0.5546875],
 'CASE37_09': [0.11198428290766209, 1.3359375],
 'CASE37_10': [0.0051933064050779, 0.0703125],
 'CASE37_11': [0.07906976744186046, 1.328125],
 'CASE37_12': [0.058823529411764705, 1.28125],
 'CASE37_13': [0.0

In [107]:
fold4

{'CASE49_36': [0.037037037037037035, 2.8125],
 'CASE49_37': [0.029292107404393815, 2.25],
 'CASE49_38': [0.04051863857374392, 3.125],
 'CASE50_01': [0.8493465634075508, 0.8568115234375],
 'CASE50_02': [0.22403733955659277, 0.1875],
 'CASE50_03': [0.31276778063410454, 0.3564453125],
 'CASE50_04': [0.009795191451469279, 0.0107421875],
 'CASE50_05': [0.3206816421378776, 0.404296875],
 'CASE50_06': [0.2619047619047619, 0.451171875],
 'CASE50_07': [0.08608205953338696, 0.208984375],
 'CASE50_08': [0.2506493506493506, 0.75390625],
 'CASE50_09': [0.2717171717171717, 1.05078125],
 'CASE50_10': [0.24808575803981622, 1.265625],
 'CASE50_11': [0.2, 1.2734375],
 'CASE50_12': [0.021415270018621976, 0.1796875],
 'CASE50_13': [0.1980495123780945, 2.0625],
 'CASE50_14': [0.03599221789883268, 0.578125],
 'CASE50_15': [0.13636363636363635, 2.625],
 'CASE50_16': [0.23922413793103448, 5.203125],
 'CASE50_17': [0.14186046511627906, 3.8125],
 'CASE50_18': [0.03467153284671533, 1.1875],
 'CASE50_19': [0.0300

In [109]:
fold5

{'CASE60_30': [0.3617021276595745, 4.9140625],
 'CASE60_31': [0.33293341331733656, 4.3359375],
 'CASE60_32': [0.1922365988909427, 2.4375],
 'CASE60_33': [0.15478366849482023, 1.984375],
 'CASE60_34': [0.01782304264799491, 0.21875],
 'CASE61_01': [0.232711966325917, 0.18896484375],
 'CASE61_02': [0.12836279784780938, 0.1630859375],
 'CASE61_03': [0.12414649286157665, 0.1953125],
 'CASE61_04': [0.1621900826446281, 0.306640625],
 'CASE61_05': [0.6113470873786407, 3.935546875],
 'CASE61_06': [0.5102040816326531, 3.515625],
 'CASE61_07': [0.5041284403669725, 4.29296875],
 'CASE61_08': [0.26911157024793386, 2.03515625],
 'CASE61_09': [0.16742700729927007, 1.43359375],
 'CASE61_10': [0.1729200652528548, 1.65625],
 'CASE61_11': [0.16666666666666666, 1.390625],
 'CASE61_12': [0.4688372093023256, 3.9375],
 'CASE61_13': [0.5078299776286354, 5.3203125],
 'CASE61_14': [0.27022900763358776, 4.1484375],
 'CASE61_15': [0.10349554489376285, 2.359375],
 'CASE61_16': [0.05857142857142857, 1.921875],
 'CA

중복데이터 삭제
case59, case11, case12, case13, case14,
이상치 리스트
확인 : CASE05_21, CASE10_41, case16, case22, case23, case45-16, case45-17,
CASE73_04, case73-10,CASE73_11, CASE73_14

In [61]:
model = LIGHTNING_MODEL(train_dataset, valid_dataset, augment_dataset, test_dataset)
trainer = pl.Trainer(
                devices=[0],
                accelerator='gpu',
                max_epochs=70,
                precision=16
            )

trainer.fit(model)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type               | Params
-------------------------------------------------------
0 | model           | CNN_RNN            | 4.4 M 
1 | train_nmae      | NMAE               | 0     
2 | valid_nmae      | NMAE               | 0     
3 | augment_dataset | PYTORCH_AUGMENTSET | 0     
4 | train_dataset   | PYTORCH_DATASET    | 0     
5 | val_dataset     | PYTORCH_DATASET    | 0     
6 | predict_dataset | PYTORCH_PREDICTSET | 0     
-------------------------------------------------------
4.4 M     Trainable params
0         Non-trainable params
4.4 M     Total params
8.707     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

validation nmae: 1.015


Training: 0it [00:00, ?it/s]

In [ ]:
#생을 돌렸을때 나오는 성능 0.1298
#mixer 사용 0.1092
